In [58]:
%%time
import pyodbc 
import pandas as pd
import numpy as np
import datetime
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import timeit

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)
#!pip install jupyter_contrib_nbextensions




ModuleNotFoundError: No module named 'numpy.testing.nosetester'

In [28]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB2;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql1 = pd.read_sql_query('SELECT TOP(1000000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df1 = pd.DataFrame(sql1)

Wall time: 32.7 s


In [29]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql2 = pd.read_sql_query('SELECT TOP(1000000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df2 = pd.DataFrame(sql2)

Wall time: 52.4 s


In [30]:
%%time
df = pd.concat([df1,df2])
df.drop_duplicates() #dropping duplicates if any

Wall time: 34.7 s


In [35]:
df.shape

(2000000, 33)

In [36]:
%%time
df['Bet Time'] = pd.to_datetime(df['Bet Time'])
df['Result Time'] = pd.to_datetime(df['Result Time'])
df['Registered Date'] = pd.to_datetime(df['Registered Date'])
df['Last Login'] = pd.to_datetime(df['Last Login'])

df['Transaction Date'] = df['Bet Time'].dt.date
df['Transaction Month'] = pd.to_datetime(df['Bet Time']).dt.to_period('M')

df['days_since_last_login'] = (df['Last Login'].max() - df['Last Login']).astype('timedelta64[D]')
df['Age_on_platform'] = (df['Last Login']- df['Registered Date']).astype('timedelta64[D]')
 
last_betting_date = df.groupby(['Player ID'])['Bet Time'].agg(['max']).reset_index().rename(columns={"max":"Last Betting Date"})
avg_no_of_bets_per_day = df.groupby(['Player ID','Transaction Date'])['Player ID'].count().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets per Day"}) #sum/count
avg_no_of_bets_per_month = df.groupby(['Player ID','Transaction Month'])['Player ID'].count().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets per Month"}) #sum/count

avg_amount_on_bets_per_day = df.groupby(['Player ID','Transaction Date'])['Stake'].sum().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets amount per Day"})
avg_amount_on_bets_per_month = df.groupby(['Player ID','Transaction Month'])['Stake'].sum().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets amount per Month"})

no_of_visiting_days = df.groupby(['Player ID', 'Transaction Date']).size().groupby('Player ID').count().reset_index().rename(columns={0:'No_of_visit_days'})

time_df = pd.DataFrame()
temp_df = df[['Player ID','days_since_last_login','Age_on_platform','Bet Time', 'Result Time','Registered Date',
       'Last Login']].groupby('Player ID').agg(['max']).reset_index() 
temp_df.columns = ['Player ID','days_since_last_login','Age_on_platform','Bet Time', 'Result Time','Registered Date','Last Login']

dataframes = [last_betting_date,avg_no_of_bets_per_day,avg_amount_on_bets_per_day,avg_no_of_bets_per_month,avg_amount_on_bets_per_month,no_of_visiting_days,temp_df]
time_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)
time_df['loyalty_ratio'] = time_df['No_of_visit_days']/time_df['Age_on_platform']


bet_df = df.groupby('Player ID').agg({
                                        "Stake": {"total_stake": "sum", "mean_stake(total_stake/total bets)": "mean", "min_stake": "min", "max_stake": "max", "bets": "count"},
                                         "Valid Bet": {"total_Valid Bet": "sum", "mean_Valid Bet total_valid bets/total bets)": "mean"},
                                        "Win/Loss": {"total_WIn/Loss": "sum", "mean_Valid Bet total_Win/Loss/total bets)": "mean"}
                                     }).reset_index()
bet_df.columns = ['Player ID','Total Stake','Avg Stake(Total Stake/Total Bets)','Min Stake','Max Stake','Total Bets','Total Valid Bet','Avg Valid Bet','Total Win/Loss', 'Avg Win/Loss']


game_group = df.groupby(['Player ID', 'Game Group']).size().reset_index() #['Game Group'].agg('count')
game_group = game_group.rename(columns ={0:'Counts'})

game_group_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_group_pref['Game Group'].unique()]
for x in temp_col1:
    game_group_pref[x]= np.where(game_group_pref['Game Group'] == x,1,0)
game_group_df = game_group_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_group_df.columns = [['Player ID']+temp_col1]

game_provider_type_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_provider_type_pref['Provider'].unique()]
for x in temp_col1:
    game_provider_type_pref[x]= np.where(game_provider_type_pref['Provider'] == x,1,0)
    
game_provider_type_pref_df = game_provider_type_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_provider_type_pref_df.columns = [['Player ID']+temp_col1]

game_product_type_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_product_type_pref['Product Type'].unique()]
for x in temp_col1:
    game_product_type_pref[x]= np.where(game_product_type_pref['Product Type'] == x,1,0)
    
game_product_type_pref_df = game_product_type_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_product_type_pref_df.columns = [['Player ID']+temp_col1]

game_aggregations_pref_fav1 = {
    'Game':{'FavFirst':[lambda x: x.value_counts().index[0] if len(x.value_counts()) else None],
           'FavFirstFrequency':[lambda x: x.value_counts()[0] if len(x.value_counts()) else None]
           } 
    } 
        
game_aggregations_pref_fav2 = {
    'Game': {'FavSecond':[lambda x: x.value_counts().index[1] if len(x.value_counts())>1 else None],
            'FavSecondFrequency':[lambda x: x.value_counts()[1] if len(x.value_counts())>1 else None]
            }
    }

game_aggregations_pref_fav3 = {
    'Game': {'FavThird':[lambda x: x.value_counts().index[2] if len(x.value_counts())>2 else None],
            'FavThirdFrequency':[lambda x: x.value_counts()[2] if len(x.value_counts())>2 else None]
            }
    } 

game_aggregations_pref_fav4 = {
    'Game': {'FavFourth':[lambda x: x.value_counts().index[3] if len(x.value_counts())>3 else None],
            'FavFourthFrequency':[lambda x: x.value_counts()[3] if len(x.value_counts())>3 else None]
            },
        } 

game_aggregations_pref_fav5 = {
    'Game': {'FavFifth':[lambda x: x.value_counts().index[4] if len(x.value_counts())>4 else None],
            'FavFifthFrequency':[lambda x: x.value_counts()[4] if len(x.value_counts())>4 else None]
            }
    } 


game_pref_1 = df.groupby('Player ID').agg(game_aggregations_pref_fav1).reset_index()
game_pref_1.columns = ['Player ID','FavFirst','FavFirstFrequency']

game_pref_2 = df.groupby('Player ID').agg(game_aggregations_pref_fav2).reset_index()
game_pref_2.columns = ['Player ID','FavSecond','FavSecondFrequency']

game_pref_3 = df.groupby('Player ID').agg(game_aggregations_pref_fav3).reset_index()
game_pref_3.columns = ['Player ID','FavThird','FavThirdFrequency']

game_pref_4 = df.groupby('Player ID').agg(game_aggregations_pref_fav4).reset_index()
game_pref_4.columns = ['Player ID','FavFourth','FavFourthFrequency']

game_pref_5 = df.groupby('Player ID').agg(game_aggregations_pref_fav5).reset_index()
game_pref_5.columns = ['Player ID','FavFifth','FavFifthFrequency']

dataframes = [game_pref_1,game_pref_2,game_pref_3,game_pref_4,game_pref_5]
game_pref_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)

game_pref_df['FavFirstAmount'] =0.0
game_pref_df['FavSecondAmount'] =0.0
game_pref_df['FavThirdAmount'] =0.0
game_pref_df['FavFourthAmount'] =0.0
game_pref_df['FavFifthAmount'] =0.0

temp_game_df = df.groupby(['Player ID','Game'])['Stake'].sum().reset_index()

for player in game_pref_df["Player ID"]:
    #print(player + "    "+ "---->")
    for game in  temp_game_df[temp_game_df['Player ID']==player]['Game']:
       # print(game)
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFirst'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFirstAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
           # print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavSecond'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavSecondAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavThird'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavThirdAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFourth'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFourthAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFifth'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFifthAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
        

dataframes = [game_product_type_pref_df,game_provider_type_pref_df,game_group_df,game_pref_df]
game_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)

dataframes = [time_df,bet_df,game_df]
final_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)


Wall time: 6min 7s


In [37]:
final_df.head()

,Player ID,Last Betting Date,Avg Bets per Day,Avg Bets amount per Day,Avg Bets per Month,Avg Bets amount per Month,No_of_visit_days,days_since_last_login,Age_on_platform,Bet Time,Result Time,Registered Date,Last Login,loyalty_ratio,Total Stake,Avg Stake(Total Stake/Total Bets),Min Stake,Max Stake,Total Bets,Total Valid Bet,Avg Valid Bet,Total Win/Loss,Avg Win/Loss,EGAME,SPORTS,LIVE,MPG_x,ANIMAL,MG,VT,SBOBET,AMEBA,MX,RT,CQ9,SA,GGAMING,PT,GD,SV,ALLBET,VENUS,SABA,UGAMING,SB,LAX,SLOTGAME,SPORTSBOOK,BACCARAT,MPG_y,DRAGON_TIGER,SICBO,FISHING,ROULETTE,CARDGAME,TABLEGAME,COCKFIGHT,FISH_PRAWN_CRAB,BLACKJACK,FavFirst,FavFirstFrequency,FavSecond,FavSecondFrequency,FavThird,FavThirdFrequency,FavFourth,FavFourthFrequency,FavFifth,FavFifthFrequency,FavFirstAmount,FavSecondAmount,FavThirdAmount,FavFourthAmount,FavFifthAmount
0,0111555214,2019-09-19 07:37:02,374.500000,902.700000,749.0,1805.40,2,154.0,653.0,2019-09-19 07:37:02,2019-09-19 07:38:37,2018-03-10 03:42:31,2019-12-23 21:57:09,0.003063,1805.40,2.410414,0.28,19.2,749,1805.4000,2.410414,-500.00,-0.667557,743,0,0,6,0,0,0,0,0,0,743,6,0,0,0,0,0,0,0,0,0,0,0,743,0,0,6,0,0,0,0,0,0,0,0,0,Golden Offer,410,Dragon's Luck,333.0,Paradise,6.0,None,NaN,None,NaN,1430.0,333.0,42.4,0.00,0.0
1,02981pleeng,2020-02-11 19:51:59,1774.200000,10343.861333,13306.5,77578.96,15,0.0,296.0,2020-02-11 19:51:59,2020-02-11 19:51:59,2019-08-03 21:58:36,2020-05-26 15:04:22,0.050676,155157.92,5.830155,0.00,329.4,26613,155083.0775,5.827343,-4900.26,-0.184130,26609,0,2,2,0,296,557,0,2599,2,21364,1525,0,0,270,0,0,0,0,0,0,0,0,26609,0,0,2,0,2,0,0,0,0,0,0,0,Winter Wonders,21125,Gemstone,1242.0,Fa Cai Shen,785.0,RuYiBang,463.0,Lucky Bat,439.0,125082.0,7762.5,3925.0,1852.00,2414.5
2,0619393970,2020-02-08 00:12:18,95.333333,276.733333,286.0,830.20,6,59.0,771.0,2020-02-08 00:12:18,2020-02-08 00:12:21,2018-02-15 00:37:25,2020-03-27 17:13:09,0.007782,1660.40,2.902797,0.30,5.0,572,1660.4000,2.902797,-797.42,-1.394091,572,0,0,0,0,262,0,0,0,0,0,310,0,0,0,0,0,0,0,0,0,0,0,572,0,0,0,0,0,0,0,0,0,0,0,0,Fa Cai Shen M,288,The Twisted Circus,262.0,1945,22.0,None,NaN,None,NaN,1440.0,110.4,110.0,0.00,0.0
3,0621620658,2019-09-15 15:14:56,25.000000,2451.090000,50.0,4902.18,2,254.0,4.0,2019-09-15 15:14:56,2019-09-15 15:15:05,2019-09-10 22:58:17,2019-09-15 15:09:39,0.500000,4902.18,98.043600,0.00,300.0,50,4812.1800,96.243600,-108.83,-2.176600,0,1,47,2,0,0,0,0,0,41,0,2,6,0,0,0,0,0,0,1,0,0,0,0,1,40,2,7,0,0,0,0,0,0,0,0,Baccarat,34,Dragon Tiger,7.0,SA Bac.10,4.0,Paradise 2,2.0,SA Bac.2,1.0,3290.0,350.0,800.0,12.18,200.0
4,0624420637,2019-09-12 12:47:43,7.000000,670.000000,7.0,670.00,1,257.0,0.0,2019-09-12 12:47:43,2019-09-12 12:48:07,2019-09-12 11:57:15,2019-09-12 12:15:36,inf,670.00,95.714286,50.00,200.0,7,660.5000,94.357143,-99.50,-14.214286,0,0,7,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,Baccarat,7,None,NaN,None,NaN,None,NaN,None,NaN,670.0,0.0,0.0,0.00,0.0


In [38]:
final_df.columns

Index(['Player ID', 'Last Betting Date', 'Avg Bets per Day',
       'Avg Bets amount per Day', 'Avg Bets per Month',
       'Avg Bets amount per Month', 'No_of_visit_days',
       'days_since_last_login', 'Age_on_platform', 'Bet Time', 'Result Time',
       'Registered Date', 'Last Login', 'loyalty_ratio', 'Total Stake',
       'Avg Stake(Total Stake/Total Bets)', 'Min Stake', 'Max Stake',
       'Total Bets', 'Total Valid Bet', 'Avg Valid Bet', 'Total Win/Loss',
       'Avg Win/Loss', 'EGAME', 'SPORTS', 'LIVE', 'MPG_x', 'ANIMAL', 'MG',
       'VT', 'SBOBET', 'AMEBA', 'MX', 'RT', 'CQ9', 'SA', 'GGAMING', 'PT', 'GD',
       'SV', 'ALLBET', 'VENUS', 'SABA', 'UGAMING', 'SB', 'LAX', 'SLOTGAME',
       'SPORTSBOOK', 'BACCARAT', 'MPG_y', 'DRAGON_TIGER', 'SICBO', 'FISHING',
       'ROULETTE', 'CARDGAME', 'TABLEGAME', 'COCKFIGHT', 'FISH_PRAWN_CRAB',
       'BLACKJACK', 'FavFirst', 'FavFirstFrequency', 'FavSecond',
       'FavSecondFrequency', 'FavThird', 'FavThirdFrequency', 'FavFourth',
      

# RFM MODELLING

Recency (R): Who have purchased recently? Number of days since last purchase (least recency)

Frequency (F): Who has purchased frequently? It means the total number of purchases. ( high frequency)

Monetary Value(M): Who have high purchase amount? It means the total money customer spent (high monetary value)

In [39]:
# from crm_rfm_modeling import rfm
# from crm_rfm_modeling.rfm import RFM

,Player ID,days_since_last_login,Total Bets,Total Stake
0,0111555214,154.0,749,1805.40
1,02981pleeng,0.0,26613,155157.92
2,0619393970,59.0,572,1660.40
3,0621620658,254.0,50,4902.18
4,0624420637,257.0,7,670.00
5,0641306590,26.0,1159,3055.98
6,0641976840,0.0,2867,4525.29
7,0806409528pop,109.0,436,508.00
8,0820295597,251.0,109,287.20
9,0824308073wan,106.0,157,166.00


In [48]:
%%time
rfm_df = final_df[['Player ID', 'days_since_last_login','Total Bets','Total Stake']]
rfm_df.columns = ['CustomerID','Recency','Frequency','Monetary']

Wall time: 2 ms


In [49]:
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary
0,0111555214,154.0,749,1805.40
1,02981pleeng,0.0,26613,155157.92
2,0619393970,59.0,572,1660.40
3,0621620658,254.0,50,4902.18
4,0624420637,257.0,7,670.00


In [50]:
CustomerID ='0619393970' # has  R59.0 F572 M1660.40 for checking

In [51]:
%%time
rfm_df['RecencyScore'] = pd.qcut(rfm_df['Recency'],5,labels=[5,4,3,2,1])
rfm_df['FrequencyScore'] = pd.qcut(rfm_df['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
rfm_df['MonetaryScore'] = pd.qcut(rfm_df['Monetary'],5,labels=[1,2,3,4,5])
rfm_df.head()

Wall time: 472 ms


In [52]:
(rfm_df['RecencyScore'].astype(str) + 
 rfm_df['FrequencyScore'].astype(str) + 
 rfm_df['MonetaryScore'].astype(str)).head()

0    344
1    555
2    444
3    224
4    113
dtype: object

In [53]:
%%time
rfm_df["RFM_SCORE"] = rfm_df['RecencyScore'].astype(str) + rfm_df['FrequencyScore'].astype(str) + rfm_df['MonetaryScore'].astype(str)

Wall time: 192 ms


In [54]:
rfm_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,937.0,139.816435,103.316161,0.0,32.00,113.0,253.0,258.0
Frequency,937.0,2134.471718,6687.940670,1.0,23.00,153.0,884.0,82328.0
Monetary,937.0,18262.930907,63451.030555,0.0,174.69,690.0,6420.0,816617.5


In [55]:
rfm_df[rfm_df["RFM_SCORE"] == "555"].head()

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
1,02981pleeng,0.0,26613,155157.92,5,5,5,555
35,2537fon,0.0,3178,217202.26,5,5,5,555
61,akira88,0.0,13861,24706.50,5,5,5,555
63,alain1313,0.0,19851,51617.94,5,5,5,555
72,angsu1608,2.0,18278,159441.00,5,5,5,555


In [56]:
rfm_df[rfm_df["RFM_SCORE"] == "111"].head()

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
70,an0620511331,256.0,5,8.00,1,1,1,111
128,bangkok2222,258.0,6,100.00,1,1,1,111
150,beerstc28,256.0,1,4.08,1,1,1,111
309,hongyok1a,257.0,1,100.00,1,1,1,111
412,kk1984,256.0,1,1.00,1,1,1,111


In [57]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [59]:
rfm_df['Segment'] = rfm_df['RecencyScore'].astype(str) + rfm_df['FrequencyScore'].astype(str)
rfm_df['Segment'] = rfm_df['Segment'].replace(seg_map, regex=True)

In [60]:
rfm_df[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])


Recency          Frequency            Monetary      
                           mean count         mean count          mean count
Segment                                                                     
About to Sleep       139.594595    74    17.486486    74    297.021081    74
At Risk              251.127660   141   336.829787   141   6436.459929   141
Can't Lose           246.166667    12  2281.833333    12   7880.246667    12
Champions              3.414966   147  9119.068027   147  66620.009524   147
Hibernating          252.715596   218    21.788991   218   1086.147477   218
Loyal Customers       89.331210   157  3564.191083   157  34263.688153   157
Need Attention       137.129630    54   148.851852    54   4269.459074    54
New Customers          8.222222     9     5.444444     9    427.144444     9
Potential Loyalists   38.130435    92   117.097826    92   4748.908043    92
Promising             51.787879    33     4.242424    33    232.353636    33

### Remark:
3 segments selected for evaluation are "Can't Lose", "Hibernating" and "Champions".

Number of customers for segments:

Can't Lose = 12, Hibernating = 218, Champions = 147

#### Can't Lose Segment;

The last shopping date of the customers is on average 246 days before.
Customers have made an average of 16 purchases.
Customers spent an average of £ 8356.

#### Hibernating Segment;

The last shopping date of the customers is 481 days before average.
Customers made an average of 1 purchases.
Customers spent an average of £ 438.

#### Champions Segment;

The last shopping date of the customers is 30 days before average.
Customers made an average of 19 purchases.
Customers spent an average of £ 10796.

#### Can't Lose Segment;

Customers in this segment have not recently made a purchase. For this reason, we need to prepare a discount and gift campaign for this segment. These customers made a large number of purchases when they made purchases before. However, recency values are lower than they should be. The campaign to be implemented for these customers should include both items purchased and recommendations based on previous activities. New and popular products associated with the products that they were interested in can also be included in this campaign. Situations that will cause these customers to stop buying need to be investigated.

#### Hibernating Segment;

Customers in this segment have not made a purchase for a long time. However, by offering discounts, they may be attracted to another purchase.

#### Champions Segment;

Customers in this segment are responsible for most of the revenue. Campaigns should be implemented to ensure the continuity of the shopping of these customers.